In [2]:
# Imports for the 
import re
import csv
import sqlite3
from pathlib import Path
import pandas as pd
import rdkit
from rdkit.Chem import Descriptors

ModuleNotFoundError: No module named 'rdkit'

In [ ]:
# Create database file and establish a connection
database_path = "./MPro_database.db"
conn = sqlite3.connect(database_path)
drop_order = ('compounds', 'assays')

In [ ]:
# Create a database of compounds and their SMILES string
conn.execute('''
CREATE TABLE compounds
(
    CID PRIMARY KEY,
    SMILES
)
        ''')

In [ ]:
df = pd.read_csv('activity_data.csv')

for i in range(len(df)):
    row = df.iloc[i]
    values  =  (row['CID'], 
         row['SMILES']
        )
    conn.execute('INSERT INTO compounds (CID, SMILES) VALUES (?,?)',
        values)

In [ ]:
# Creates and populates a table of assays, including a foreign key linking the assays to the compounds table
conn.execute('''
CREATE TABLE assays
(
    CID PRIMARY KEY,
    canonical_CID,
    r_inhibition_at_20_uM,
    r_inhibition_at_50_uM,
    r_avg_IC50,
    f_inhibition_at_20_uM,
    f_inhibition_at_50_uM,
    f_avg_IC50,
    f_avg_pIC50,
    relative_solubility_at_20_uM,
    relative_solubility_at_100_uM,
    trypsin_IC50,
    NMR_std_ratio,
    acrylamide,
    chloroacetamide,
    series,
    frag_id,
    compoundID,
    FOREIGN KEY (CompoundID) REFERENCES compounds (CID)
)
        ''')


In [ ]:
df_short = df.iloc[:,1:]

for i in range(len(df_short)):
    row = df_short.iloc[i]
    values = []
    
    for element in row[:]:
        values.append(element)
    values.append(row[0])
    conn.execute('INSERT INTO assays (CID, canonical_CID, r_inhibition_at_20_uM, r_inhibition_at_50_uM, r_avg_IC50, f_inhibition_at_20_uM, f_inhibition_at_50_uM, f_avg_IC50, f_avg_pIC50, relative_solubility_at_20_uM, relative_solubility_at_100_uM, trypsin_IC50, NMR_std_ratio, acrylamide, chloroacetamide, series, frag_id, compoundID) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
             (values)
                )

In [ ]:
def retrieve_SMILES(compoundID: str):
    list_smiles = []
    SMILES = conn.execute('''
        SELECT SMILES
        FROM compounds
        WHERE 
            CID = ?
    ''', (compoundID,))
    for sm in SMILES:
        list_smiles.append(sm)
    return (list_smiles[0])
        
def retrieve_assays(compoundID: str):
    list_assays = []
    ASSAYS = conn.execute('''
        SELECT *
        FROM assays
        WHERE 
            compoundID = ?
    ''', (compoundID,))
    
    for assay in ASSAYS:
        list_assays.append(assay)
    return (list_assays[0][2:-1])

In [ ]:
# Demonstrate we can retrieve the assay data and SMILES string from an ID

print(retrieve_SMILES('MAT-POS-be048f2c-5'))
print(retrieve_assays('MAT-POS-be048f2c-5'))

('CCn1nccc1C(=O)N1Cc2ccc(Cl)cc2C2(CCN(c3cncc4ccccc34)C2=O)C1',)
(None, None, None, None, None, 0.297012069374338, None, None, None, None, None, 0, 0, '3-aminopyridine-like', None)


In [ ]:
# Code to display the tables currently in the database

"""
cur = conn.execute('''
            SELECT 
                name
            FROM 
                sqlite_master 
            WHERE 
                type ='table' AND 
                name NOT LIKE 'sqlite_%'
        ''')

tables = []

for row in cur.fetchall():
    tables.append(row[0])

for table in tables:
    cur = conn.execute('SELECT  sql  FROM  sqlite_master  WHERE name=\'' + table + '\'')

    row = cur.fetchone()

    if row is not None:
        print(table+'\n')

        print(row)
    else:
        print('Table ' + table + ' not found')
"""

compounds

('CREATE TABLE compounds\n(\n    CID PRIMARY KEY,\n    SMILES\n)',)
assays

('CREATE TABLE assays\n(\n    CID PRIMARY KEY,\n    canonical_CID,\n    r_inhibition_at_20_uM,\n    r_inhibition_at_50_uM,\n    r_avg_IC50,\n    f_inhibition_at_20_uM,\n    f_inhibition_at_50_uM,\n    f_avg_IC50,\n    f_avg_pIC50,\n    relative_solubility_at_20_uM,\n    relative_solubility_at_100_uM,\n    trypsin_IC50,\n    NMR_std_ratio,\n    acrylamide,\n    chloroacetamide,\n    series,\n    frag_id,\n    compoundID,\n    FOREIGN KEY (CompoundID) REFERENCES compounds (CID)\n)',)


In [ ]:
#for table_name in drop_order:
 #   conn.execute('DROP TABLE IF EXISTS ' + table_name)